<a href="https://colab.research.google.com/github/murilo-henrique060/matematica-computacional/blob/main/Atividade%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [553]:
from IPython.display import HTML

import itertools, math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [532]:
class Plot():
  def __init__(self, grid=(1, 1), figsize=(4, 4), axis=(0, 4, 0, 4)):
    self.fig, self.ax = plt.subplots(grid[0], grid[1], figsize=figsize)
    self.fig.tight_layout()
    self._updated = []

    if grid[0] == 1 and grid[1] == 1:
      self.ax._plot = self
      self.ax.grid(True)
      self.ax.set_xlim(axis[0], axis[1])
      self.ax.set_ylim(axis[2], axis[3])
      return

    for index, ax in enumerate(self.ax):
      ax._plot = self
      ax.grid(True)
      ax.set_xlim(axis[index][0], axis[index][1])
      ax.set_ylim(axis[index][2], axis[index][3])

  @property
  def updated(self):
    updated = self._updated
    self._updated = []
    return updated

  def show(self):
    plt.show()

  def animate(self, update, frames=100, blit=True, interval=100, **kwargs):
    plt.close()
    self._updated.clear()
    self.ani = animation.FuncAnimation(self.fig, update, frames=frames, blit=blit, interval=interval, **kwargs)
    return HTML(self.ani.to_html5_video())

In [533]:
class PlotObject():
  def __init__(self, ax, points, zorder=1, **kwargs):
    self.ax = ax
    self.line, = ax.plot((), (), zorder=zorder+1000, **kwargs)
    self.points = points

  @property
  def points(self):
    return np.array(list(zip(*self.line.get_data())))

  @points.setter
  def points(self, points):
    if len(points) == 0:
      self.line.set_data((), ())
    else:
      self.line.set_data(*zip(*points))

    if (self.line not in self.ax._plot._updated):
      self.ax._plot._updated.append(self.line)

In [534]:
class PointObject(PlotObject):
  @staticmethod
  def factory(ax, points, **kwargs):
    return [PointObject(ax, point, **kwargs) for point in points]

  def __init__(self, ax, point, color='green', marker='o', **kwargs):
    super().__init__(ax, [point], color=color, marker=marker, **kwargs)

  @property
  def point(self):
    return self.points[0]

  @point.setter
  def point(self, point):
    self.points = [point]

  @property
  def x(self):
    return self.point[0]

  @x.setter
  def x(self, x):
    self.point = (x, self.point[1])

  @property
  def y(self):
    return self.point[1]

  @y.setter
  def y(self, y):
    self.point = (self.point[0], y)

In [535]:
class LineObject(PlotObject):
  @staticmethod
  def factory(ax, lines, **kwargs):
    return [LineObject(ax, *line, **kwargs) for line in lines]

  def __init__(self, ax, start, end, color='gray', **kwargs):
    super().__init__(ax, [start, end], color=color, **kwargs)

  @property
  def start(self):
    return self.points[0]

  @start.setter
  def start(self, start):
    self.points = [start, self.points[1]]

  @property
  def end(self):
    return self.points[1]

  @end.setter
  def end(self, end):
    self.points = [self.points[0], end]

In [536]:
class FunctionObject(PlotObject):
  def __init__(self, ax, function=lambda x: x, samples=100, color='blue', **kwargs):
    self._function = function
    self._samples = samples
    super().__init__(ax, [], color=color, **kwargs)

    self.update_path()

  def update_path(self):
    xlimits = self.ax.get_xlim()
    self.xpoints = np.linspace(xlimits[0], xlimits[1], self._samples)
    self.line.set_data(self.xpoints, list(map(self._function, self.xpoints)))

  @property
  def function(self):
    return self._function

  @function.setter
  def function(self, function):
    self._function = function
    self.update_path()

  @property
  def samples(self):
    return self._samples

  @samples.setter
  def samples(self, samples):
    self._samples = samples
    self.update_path()

In [537]:
class LineFunctionObject(FunctionObject):
  def __init__(self, plot, function=lambda x: x, **kwargs):
    super().__init__(plot, function, samples=2, **kwargs)

In [538]:
class LinearRegressionIntercept():
  def __init__(self, slope=1, intercept=0):
    self.slope = slope
    self.intercept = intercept

  def predict(self, x, slope=None, intercept=None):
    if slope is None:
      slope = self.slope
    if intercept is None:
      intercept = self.intercept

    return slope * x + intercept

  def descent_gradient(self, data, **kwargs):
    residual_squared_sum = 0
    gradient = 0

    for x, y in data:
      diff = y - self.predict(x, **kwargs)
      residual_squared_sum += diff**2
      gradient += -2 * diff

    return residual_squared_sum, gradient

  def learn(self, data, learn_rate=0.1, max_reps=100, min_step=0.001, snapshots=False, verbose=False):
    for _ in range(max_reps):
      residual_squared_sum, loss = self.descent_gradient(data)

      step = learn_rate * loss
      new_intercept = self.intercept - step

      if verbose:
        print(f"step: {step:<25} intercept: {self.intercept:<25} new intercept: {new_intercept}")

      if snapshots:
        yield self.intercept, residual_squared_sum, loss

      self.intercept = new_intercept

      if abs(step) <= min_step:
        return


In [539]:
DATA = [
  (0.5, 1.4),
  (2.3, 1.9),
  (2.9, 3.2)
]

In [540]:
def animated_training_intercept(data, max_reps=100, **kwargs):
  linear_regression = LinearRegressionIntercept(slope=0.64)

  # Defining plot
  plot = Plot(grid=(1, 2), figsize=(8, 4), axis=[(0, 3, 0, 4), (-1, 3, 0, 8)])

  # Setup left
  data_points = PointObject.factory(plot.ax[0], DATA, zorder=2) # Data points plot in green
  diff_bars = LineObject.factory(plot.ax[0], map(lambda point: (point, point), DATA), zorder=1) # Differencence bars plotted in gray
  predict_function = LineFunctionObject(plot.ax[0], color='lightblue', zorder=2) # Predict function plotted in light blue

  # Setup right
  loss_function = FunctionObject(plot.ax[1], lambda x: linear_regression.descent_gradient(DATA, intercept=x)[0], color='blue') # Loss in function of bias plotted in blue
  gradient_line = LineFunctionObject(plot.ax[1], color='green') # Gradient line plotted in green

  # Display training
  def update(data):
    intercept, residual_squared_sum, loss = data

    # Updating left
    predict_function.function = lambda x: linear_regression.predict(x, intercept=intercept) # Updating predict function
    for bar in diff_bars:
      end = bar.end
      end[1] = linear_regression.predict(end[0], intercept=intercept) # Updating each diff bar possition
      bar.end = end

    # Updating right
    gradient_line.function = lambda x: loss * (x - intercept) + residual_squared_sum # Updating the gradient line based on loss function using y - y0 = m(x - x0) => y = m(x - x0) + y0
    PointObject(plot.ax[1], (intercept, residual_squared_sum), color='red') # Plot point representing current b

    return plot.updated

  # Setup training
  train_iterator = linear_regression.learn(DATA, max_reps=max_reps, snapshots=True, verbose=True, **kwargs)
  return plot.animate(update, frames=train_iterator, save_count=max_reps)

In [541]:
animated_training_intercept(DATA, learn_rate=0.1)

step: -0.5704000000000001       intercept: 0                         new intercept: 0.5704000000000001
step: -0.22815999999999992      intercept: 0.5704000000000001        new intercept: 0.79856
step: -0.09126399999999997      intercept: 0.79856                   new intercept: 0.8898240000000001
step: -0.03650560000000005      intercept: 0.8898240000000001        new intercept: 0.9263296000000001
step: -0.014602239999999968     intercept: 0.9263296000000001        new intercept: 0.9409318400000001
step: -0.005840895999999951     intercept: 0.9409318400000001        new intercept: 0.946772736
step: -0.002336358400000016     intercept: 0.946772736               new intercept: 0.9491090944
step: -0.0009345433600000064    intercept: 0.9491090944              new intercept: 0.95004363776


In [542]:
animated_training_intercept(DATA, learn_rate=0.01)

step: -0.05704000000000001      intercept: 0                         new intercept: 0.05704000000000001
step: -0.0536176                intercept: 0.05704000000000001       new intercept: 0.11065760000000001
step: -0.050400544000000005     intercept: 0.11065760000000001       new intercept: 0.16105814400000001
step: -0.047376511360000005     intercept: 0.16105814400000001       new intercept: 0.20843465536000003
step: -0.0445339206784          intercept: 0.20843465536000003       new intercept: 0.2529685760384
step: -0.041861885437696        intercept: 0.2529685760384           new intercept: 0.294830461476096
step: -0.03935017231143424      intercept: 0.294830461476096         new intercept: 0.3341806337875302
step: -0.03698916197274819      intercept: 0.3341806337875302        new intercept: 0.3711697957602784
step: -0.0347698122543833       intercept: 0.3711697957602784        new intercept: 0.4059396080146617
step: -0.0326836235191203       intercept: 0.4059396080146617        new 

In [560]:
class LinearRegression():
  def __init__(self, slope=1, intercept=0):
    self.slope = slope
    self.intercept = intercept

  def predict(self, x, slope=None, intercept=None):
    if slope is None:
      slope = self.slope
    if intercept is None:
      intercept = self.intercept

    return slope * x + intercept

  def descent_gradient(self, data, **kwargs):
    residual_squared_sum = 0

    slope_gradient = 0
    intercept_gradient = 0

    for x, y in data:
      diff = y - self.predict(x, **kwargs)
      residual_squared_sum += diff**2

      slope_gradient += -2 * x * diff
      intercept_gradient += -2 * diff

    return residual_squared_sum, slope_gradient, intercept_gradient

  def learn(self, data, learn_rate=0.1, max_reps=100, min_step=0.001, snapshots=False, verbose=False):
    for i in range(max_reps):
      residual_squared_sum, slope_loss, intercept_loss = self.descent_gradient(data)

      slope_step = learn_rate * slope_loss
      new_slope = self.slope - slope_step

      intercept_step = learn_rate * intercept_loss
      new_intercept = self.intercept - intercept_step

      if verbose:
        print(f"iteration: {i:<10} slope: {self.slope:<30} slope step: {slope_step:<30} intercept: {self.intercept:<30} intercept step: {intercept_step:<30}")

      if snapshots:
        yield self.slope, self.intercept, residual_squared_sum, slope_loss, intercept_loss

      self.slope = new_slope
      self.intercept = new_intercept

      if abs(slope_step) < min_step and abs(intercept_step) < min_step:
        return

In [561]:
def animated_training(data, max_reps=100, interval=50, **kwargs):
  linear_regression = LinearRegression(slope=1, intercept=0)

  # Defining plot
  plot = Plot(grid=(1, 1), figsize=(4, 4), axis=(0, 3, 0, 4))

  data_points = PointObject.factory(plot.ax, data, zorder=2),
  diff_bars = LineObject.factory(plot.ax, zip(data, data), zorder=1)
  predict_function = LineFunctionObject(plot.ax, color='lightblue', zorder=2)

  # Display training
  def update(data):
    slope, intercept, residual_squared_sum, slope_loss, intercept_loss = data

    predict_function.function = lambda x: linear_regression.predict(x, slope=slope, intercept=intercept) # Updating predict function
    for bar in diff_bars:
      end = bar.end
      end[1] = linear_regression.predict(end[0], slope=slope, intercept=intercept) # Updating each diff bar possition
      bar.end = end

    return plot.updated

  # Setup training
  train_iterator = linear_regression.learn(data, max_reps=max_reps, snapshots=True, verbose=True, **kwargs)
  return plot.animate(update, frames=train_iterator, save_count=max_reps, interval=interval)




In [563]:
animated_training(DATA, learn_rate=0.05)

iteration: 0          slope: 1                              slope step: -0.040000000000000105          intercept: 0                              intercept step: -0.08000000000000003          
iteration: 1          slope: 1.04                           slope step: 0.06139999999999999            intercept: 0.08000000000000003            intercept step: -0.033199999999999986         
iteration: 2          slope: 0.9786                         slope step: -0.005329000000000062          intercept: 0.11320000000000002            intercept step: -0.05823800000000001          
iteration: 3          slope: 0.983929                       slope step: 0.03530061500000005            intercept: 0.17143800000000003            intercept step: -0.03772906999999997          
iteration: 4          slope: 0.948628385                    slope step: 0.007561826974999797           intercept: 0.20916707                     intercept step: -0.046531699550000076         
iteration: 5          slope: 0.941066558

In [552]:
animated_training(DATA, learn_rate=0.001, max_reps=10000, min_step=0.0001, interval=1)

slope: 1                              slope step: -0.0008000000000000021         intercept: 0                              intercept step: -0.0016000000000000005        
slope: 1.0008                         slope step: -0.0007594400000000077         intercept: 0.0016000000000000005          intercept step: -0.0015812800000000026        
slope: 1.0015594399999999             slope step: -0.0007202250320000032         intercept: 0.0031812800000000033          intercept step: -0.0015631347040000012        
slope: 1.0022796650319998             slope step: -0.0006823110179816042         intercept: 0.004744414704000005           intercept step: -0.0015455453304112014        
slope: 1.0029619760499815             slope step: -0.0006456553238132279         intercept: 0.006289960034411206           intercept step: -0.0015284937128237433        
slope: 1.0036076313737947             slope step: -0.0006102167119526534         intercept: 0.00781845374723495            intercept step: -0.00151196